In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Inception 블록

In [2]:
class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_1x1, reduce_3x3, out_3x3, reduce_5x5, out_5x5, out_pool):
        super(InceptionModule, self).__init__()
        
        # 1x1 convolution branch
        self.conv1x1 = nn.Conv2d(in_channels, out1x1, kernel_size=1)
        
        # 3x3 convolution branch
        self.conv3x3_reduce = nn.Conv2d(in_channels, reduce_3x3, kernel_size=1) # 입력 채널을 줄여줌
        self.conv3x3 = nn.Conv2d(reduce3x3, out_3x3, kernel_size=3, padding=1)
        
        
        # 5x5 convolution branch
        self.conv5x5_reduce = nn.Conv2d(in_channels, reduce_5x5, kernel_size=1)
        self.conv5x5 = nn.Conv2d(reduce_5x5, out_5x5, kernel_size=5, padding=2)
        
        # max pooling branch
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.conv_pool = nn.Conv2d(in_channels, out_pool, kernel_size=1)
        
    def forward(self, x):
        # 1x1 conv
        out_1x1 = F.relu(self.conv1x1(x))
        
        # 3x3 conv
        out_3x3 = F.relu(self.conv3x3_reduce(x))
        out_3x3 = F.relu(self.conv3x3(out_3x3))
        
        # 5x5 conv
        out_5x5 = F.relu(self.conv5x5_reduce(x))
        out_5x5 = F.relu(self.conv5x5(out_5x5))
        
        # Max pooling
        out_pool = F.relu(self.conv_pool(self.pool(x)))
        
        # Concatenate all 
        out = torch.cat([out_1x1, out_3x3, out_5x5, out_pool], dim=1)
        
        return out

# Inception 모델 정의

In [3]:
class InceptionV2(nn.Module):
    def __init__(self, num_classes=1000):
        super(InceptionV2, self).__init__()
        
        # Stem layers
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3), # padding=3 >> 가장자리 값의 정보 손실 방지
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # padding=1 >> 가장자리 값의 정보 손실 방지
        )
        # Inception module
        self.inception3a = InceptionModule(64, 64, 96, 128, 16, 32, 32) 
        self.inception3b = InceptionModule(256, 128, 128, 192, 32, 96, 64)
        self.inception4a = InceptionModule(480, 192, 96, 203, 16, 48, 64)
        self.inception4b = InceptionModule(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionModule(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionModule(512, 112, 144, 288, 32, 64, 64)  
        self.inception4e = InceptionModule(528, 256, 160, 320, 32, 128, 128)
        self.inception5a = InceptionModule(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionModule(832, 384, 192, 384, 48, 128, 128)
        
        # avg pool
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        
        # Stem layer
        x = self.stem(x)
        
        # inception
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.inception5a(x)
        x = self.inception5b(x)
        
        # avg pool
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x
        